In [2]:
# coco_vg_info: list of info for images that are in vg and coco
import json
from pprint import pprint
with open('coco_overlap.json') as data_file:
    coco_vg_info = json.load(data_file)
print(len(coco_vg_info))

51498


In [3]:
# vg_idx: vg image that also in coco
vg_idx = []
for dic in coco_vg_info:
    vg_idx.append(int(dic['image_id']))
vg_idx.sort()
print(len(vg_idx))                     

51498


In [4]:
print(vg_idx[0])

61512


# h5 format
> **key: gt**   
>> group key <HDF5 group "/gt/test" (25858 members)> <br/>
>> group key <HDF5 group "/gt/train" (73794 members)> <br/>

|   key       |          format                        |  
|-------------|:--------------------------------------:|  
| obj_boxes   | n\*4 array ($x_1$, $y_1$, $x_2$, $y_2$) ?|     
| rlp_labels  | n\*3 array (s,p,o)                     |
| sub_boxes   | n\*4 array ($x_1$, $y_1$, $x_2$, $y_2$)|  

> **key: meta**
>> group key <HDF5 group "/meta/cls" (2 members)> <br/>
>>> idx2name(201, from 0)  
>>> name2idx(201, from 0)  

>>group key <HDF5 group "/meta/imid2path" (108077 members)> <br/>
>>> img path('VG_100K_2/1.jpg')

>> group key <HDF5 group "/meta/pre" (2 members)> <br/>
>>> idx2name(100, from 0)

In [102]:
import h5py
import os
import bisect
import numpy as np

In [145]:
path=['/gt/test/', '/gt/train/']
test = 0
train = 0
print(os.path.getsize('vg1_2_meta2.h5'))

278024180


In [146]:
deleted=0
with h5py.File('vg1_2_meta2.h5','a') as f:
    for key in f[path[0]].keys():
        test +=1
        int_key = int(key)
        # test whether key is in vg_idx
        idx = bisect.bisect(vg_idx, int_key)
        if vg_idx[idx-1]!=int_key: # not in vg_idx
            # delete that group
            del f[path[0]+str(key)]
            deleted = deleted + 1
#         if key not in vg_idx:
#             del f[path[0]+str(key)]
#         f.__delitem__('gt/train/2391709')
#         print(f['gt/train/2391709'])
print(deleted)

13473


In [147]:
deleted=0
with h5py.File('vg1_2_meta2.h5','a') as f:
    for key in f[path[1]].keys():
        test +=1
        int_key = int(key)
        # test whether key is in vg_idx
        idx = bisect.bisect(vg_idx, int_key)
        if vg_idx[idx-1]!=int_key: # not in vg_idx
            # delete that group
            del f[path[1]+str(key)]
            deleted = deleted + 1
#         if key not in vg_idx:
#             del f[path[0]+str(key)]
#         f.__delitem__('gt/train/2391709')
#         print(f['gt/train/2391709'])
print(deleted)

38732


In [48]:
f.close()
print(os.path.getsize('vg1_2_meta2.h5'))

278024180


In [55]:
paths=['/gt/test','/gt/train', '/meta/cls','/meta/imid2path','/meta/pre']
fs = h5py.File('vg1_2_meta2.h5', 'r')
fd = h5py.File('cleaned_vg.h5','w')
for path in paths:
    group_path = fs[path].parent.name
    group_id = fd.require_group(group_path)
    fs.copy(path, group_id, name=path.split('/')[-1])

In [57]:
for path in paths:
    print(fd[path])

<HDF5 group "/gt/test" (12385 members)>
<HDF5 group "/gt/train" (35062 members)>
<HDF5 group "/meta/cls" (2 members)>
<HDF5 group "/meta/imid2path" (108077 members)>
<HDF5 group "/meta/pre" (2 members)>


In [58]:
fs.close()
fd.close()

In [189]:
predicate_list = [] # list of tuple: (id, name)
predicate_id2name_dic = {}
predicate_name2id_dic = {}
f = h5py.File('cleaned_vg.h5','r')
idx_list = []
name_list = []
for item in f['meta/pre/idx2name']:
    idx_list.append(int(item))
    name_list.append(f['meta/pre/idx2name/'+str(item)].value.lower())
for idx, name in zip(idx_list, name_list):
    predicate_list.append((idx,name))
    predicate_id2name_dic[idx] = name
    predicate_name2id_dic[name] = idx

In [190]:
print(len(predicate_list))
print(predicate_list[0])

100
(0, 'above')


In [191]:
class_list = [] # list of tuple: (id, name)
class_id2name_dict = {}
class_name2id_dict = {}
idx_list = []
name_list = []
for item in f['meta/cls/idx2name']:
    idx_list.append(int(item))
    name_list.append(f['meta/cls/idx2name/'+str(item)].value.lower())
name_list[0] = '__background__'
for idx, name in zip(idx_list, name_list):
    class_list.append((idx,name))
    class_id2name_dict[idx] = name
    class_name2id_dict[name] = idx

In [192]:
print(class_id2name_dict)

{0: '__background__', 1: 'frisbee', 10: 'basket', 100: 'laptop', 101: 'leaf', 102: 'leg', 103: 'letter', 104: 'light', 105: 'line', 106: 'logo', 107: 'mirror', 108: 'motorcycle', 109: 'mountain', 11: 'bat', 110: 'mouth', 111: 'napkin', 112: 'neck', 113: 'nose', 114: 'number', 115: 'ocean', 116: 'orange', 117: 'pant', 118: 'paper', 119: 'part', 12: 'bathroom', 120: 'pavement', 121: 'paw', 122: 'people', 123: 'person', 124: 'phone', 125: 'picture', 126: 'pillow', 127: 'pizza', 128: 'plane', 129: 'plant', 13: 'beach', 130: 'plate', 131: 'platform', 132: 'player', 133: 'pole', 134: 'pot', 135: 'racket', 136: 'reflection', 137: 'road', 138: 'rock', 139: 'roof', 14: 'bear', 140: 'room', 141: 'sandwich', 142: 'seat', 143: 'shadow', 144: 'sheep', 145: 'shelf', 146: 'shirt', 147: 'shoe', 148: 'short', 149: 'sidewalk', 15: 'bed', 150: 'sign', 151: 'sink', 152: 'skateboard', 153: 'skateboarder', 154: 'ski', 155: 'skier', 156: 'sky', 157: 'snow', 158: 'sock', 159: 'spot', 16: 'bench', 160: 'stand'

In [193]:
print(len(class_list))
print(class_list[0])

201
(0, '__background__')


In [194]:
predicate_list.sort()
print(predicate_list)

[(0, 'above'), (1, 'across'), (2, 'adorn'), (3, 'against'), (4, 'along'), (5, 'around'), (6, 'at'), (7, 'attach to'), (8, 'behind'), (9, 'belong to'), (10, 'below'), (11, 'beneath'), (12, 'beside'), (13, 'between'), (14, 'build into'), (15, 'by'), (16, 'carry'), (17, 'cast'), (18, 'catch'), (19, 'connect to'), (20, 'contain'), (21, 'cover'), (22, 'cover in'), (23, 'cover with'), (24, 'cross'), (25, 'cut'), (26, 'drive on'), (27, 'eat'), (28, 'face'), (29, 'fill with'), (30, 'fly'), (31, 'fly in'), (32, 'for'), (33, 'from'), (34, 'grow in'), (35, 'grow on'), (36, 'hang in'), (37, 'hang on'), (38, 'have'), (39, 'hit'), (40, 'hold'), (41, 'hold by'), (42, 'in'), (43, 'in front of'), (44, 'in middle of'), (45, 'inside'), (46, 'lay in'), (47, 'lay on'), (48, 'lean on'), (49, 'look at'), (50, 'mount on'), (51, 'near'), (52, 'next to'), (53, 'of'), (54, 'on'), (55, 'on back of'), (56, 'on bottom of'), (57, 'on side of'), (58, 'on top of'), (59, 'outside'), (60, 'over'), (61, 'paint on'), (62,

In [195]:
class_list.sort()
print(class_list)

[(0, '__background__'), (1, 'frisbee'), (2, 'air'), (3, 'animal'), (4, 'arm'), (5, 'back'), (6, 'backpack'), (7, 'bag'), (8, 'ball'), (9, 'banana'), (10, 'basket'), (11, 'bat'), (12, 'bathroom'), (13, 'beach'), (14, 'bear'), (15, 'bed'), (16, 'bench'), (17, 'bicycle'), (18, 'bike'), (19, 'bird'), (20, 'blanket'), (21, 'board'), (22, 'boat'), (23, 'book'), (24, 'bottle'), (25, 'bowl'), (26, 'box'), (27, 'boy'), (28, 'branch'), (29, 'bread'), (30, 'brick'), (31, 'bridge'), (32, 'broccoli'), (33, 'building'), (34, 'bus'), (35, 'bush'), (36, 'button'), (37, 'cabinet'), (38, 'cake'), (39, 'car'), (40, 'cat'), (41, 'catcher'), (42, 'ceiling'), (43, 'chair'), (44, 'cheese'), (45, 'clock'), (46, 'cloth'), (47, 'cloud'), (48, 'collar'), (49, 'container'), (50, 'couch'), (51, 'counter'), (52, 'court'), (53, 'cow'), (54, 'curtain'), (55, 'design'), (56, 'desk'), (57, 'dirt'), (58, 'dog'), (59, 'door'), (60, 'dough nut'), (61, 'ear'), (62, 'edge'), (63, 'elephant'), (64, 'engine'), (65, 'eye'), (6

In [196]:
class_list[115] = (115, 'sea')
class_list[60] = (60, 'donut')
class_list[18] = (18, 'bicycle')
class_list[9] = (9, 'pavement')
class_list[12] = (12, 'toilet')
class_list[101] = (101, 'leaves')
class_list[13] = (13, 'sand')
print(class_list)

[(0, '__background__'), (1, 'frisbee'), (2, 'air'), (3, 'animal'), (4, 'arm'), (5, 'back'), (6, 'backpack'), (7, 'bag'), (8, 'ball'), (9, 'pavement'), (10, 'basket'), (11, 'bat'), (12, 'toilet'), (13, 'sand'), (14, 'bear'), (15, 'bed'), (16, 'bench'), (17, 'bicycle'), (18, 'bicycle'), (19, 'bird'), (20, 'blanket'), (21, 'board'), (22, 'boat'), (23, 'book'), (24, 'bottle'), (25, 'bowl'), (26, 'box'), (27, 'boy'), (28, 'branch'), (29, 'bread'), (30, 'brick'), (31, 'bridge'), (32, 'broccoli'), (33, 'building'), (34, 'bus'), (35, 'bush'), (36, 'button'), (37, 'cabinet'), (38, 'cake'), (39, 'car'), (40, 'cat'), (41, 'catcher'), (42, 'ceiling'), (43, 'chair'), (44, 'cheese'), (45, 'clock'), (46, 'cloth'), (47, 'cloud'), (48, 'collar'), (49, 'container'), (50, 'couch'), (51, 'counter'), (52, 'court'), (53, 'cow'), (54, 'curtain'), (55, 'design'), (56, 'desk'), (57, 'dirt'), (58, 'dog'), (59, 'door'), (60, 'donut'), (61, 'ear'), (62, 'edge'), (63, 'elephant'), (64, 'engine'), (65, 'eye'), (66,

In [197]:
coco_obj_categories = [] # store all coco object classes that have segmentation
coco_stuff_categories = [] # store all coco stull classes that have segmentation
with open('coco_obj_categories.txt','r') as f:
    coco_obj_categories = f.readlines()
with open('coco_stuff_categories.txt','r') as f:
    coco_stuff_categories = f.readlines()

In [198]:
for i,line in enumerate(coco_obj_categories):
    coco_obj_categories[i] = line.replace('\n','')
print(coco_obj_categories)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [199]:
for i,line in enumerate(coco_stuff_categories):
    coco_stuff_categories[i] = line.replace('\n','')
    coco_stuff_categories[i] = coco_stuff_categories[i].lower()
print(coco_stuff_categories)

['banner', 'blanket', 'branch', 'bridge', 'building-other', 'bush', 'cabinet', 'cage', 'cardboard', 'carpet', 'ceiling-other', 'ceiling-tile', 'cloth', 'clothes', 'clouds', 'counter', 'cupboard', 'curtain', 'desk-stuff', 'dirt', 'door-stuff', 'fence', 'floor-marble', 'floor-other', 'floor-stone', 'floor-tile', 'floor-wood', 'flower', 'fog', 'food-other', 'fruit', 'furniture-other', 'grass', 'gravel', 'ground-other', 'hill', 'house', 'leaves', 'light', 'mat', 'metal', 'mirror-stuff', 'moss', 'mountain', 'mud', 'napkin', 'net', 'paper', 'pavement', 'pillow', 'plant-other', 'plastic', 'platform', 'playingfield', 'railing', 'railroad', 'river', 'road', 'rock', 'roof', 'rug', 'salad', 'sand', 'sea', 'shelf', 'sky-other', 'skyscraper', 'snow', 'solid-other', 'stairs', 'stone', 'straw', 'structural-other', 'table', 'tent', 'textile-other', 'towel', 'tree', 'vegetable', 'wall-brick', 'wall-concrete', 'wall-other', 'wall-panel', 'wall-stone', 'wall-tile', 'wall-wood', 'water-other', 'waterdrops

In [200]:
vg_obj_categories = []
for cls in class_list:
    vg_obj_categories.append(cls[1])
print(vg_obj_categories)

['__background__', 'frisbee', 'air', 'animal', 'arm', 'back', 'backpack', 'bag', 'ball', 'pavement', 'basket', 'bat', 'toilet', 'sand', 'bear', 'bed', 'bench', 'bicycle', 'bicycle', 'bird', 'blanket', 'board', 'boat', 'book', 'bottle', 'bowl', 'box', 'boy', 'branch', 'bread', 'brick', 'bridge', 'broccoli', 'building', 'bus', 'bush', 'button', 'cabinet', 'cake', 'car', 'cat', 'catcher', 'ceiling', 'chair', 'cheese', 'clock', 'cloth', 'cloud', 'collar', 'container', 'couch', 'counter', 'court', 'cow', 'curtain', 'design', 'desk', 'dirt', 'dog', 'door', 'donut', 'ear', 'edge', 'elephant', 'engine', 'eye', 'fence', 'field', 'finger', 'flag', 'floor', 'flower', 'food', 'foot', 'fork', 'frame', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'grass', 'ground', 'hair', 'hand', 'handle', 'hat', 'head', 'headlight', 'helmet', 'horn', 'horse', 'house', 'hydrant', 'jacket', 'keyboard', 'kitchen', 'kite', 'knife', 'lamp', 'laptop', 'leaves', 'leg', 'letter', 'light', 'line', 'logo', 'mirror', 'motor

In [201]:
final_object_list = [] # any object not in this list will be filtered out
for obj in vg_obj_categories:
    if any(obj in s for s in coco_obj_categories) or any(obj in s for s in coco_stuff_categories):
        final_object_list.append(obj)


In [202]:
print(final_object_list)

['frisbee', 'air', 'back', 'backpack', 'bag', 'ball', 'pavement', 'bat', 'toilet', 'sand', 'bear', 'bed', 'bench', 'bicycle', 'bicycle', 'bird', 'blanket', 'board', 'boat', 'book', 'bottle', 'bowl', 'branch', 'brick', 'bridge', 'broccoli', 'building', 'bus', 'bush', 'cabinet', 'cake', 'car', 'cat', 'ceiling', 'chair', 'clock', 'cloth', 'cloud', 'couch', 'counter', 'cow', 'curtain', 'desk', 'dirt', 'dog', 'door', 'donut', 'ear', 'elephant', 'fence', 'field', 'floor', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'glass', 'glove', 'grass', 'ground', 'hair', 'hand', 'horse', 'house', 'hydrant', 'keyboard', 'kite', 'knife', 'laptop', 'leaves', 'light', 'mirror', 'motorcycle', 'mountain', 'napkin', 'sea', 'orange', 'paper', 'pavement', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'platform', 'pot', 'racket', 'road', 'rock', 'roof', 'sandwich', 'sheep', 'shelf', 'sign', 'sink', 'skateboard', 'ski', 'sky', 'snow', 'suitcase', 'surfboard', 'table', 'teddy bear', 'tie', 'tile', 'toil

In [222]:
deleted=0
test = 0
paths=['/gt/test/', '/gt/train/']
with h5py.File('filtered_vg.h5','a') as f:
#     for group in f.keys():
#         print(group)
    for path in paths:
        for key in f[path].keys():
            test += 1
            obj = f[path+str(key)+'/obj_boxes'].value
            arr = f[path+str(key)+'/rlp_labels'].value
            sub = f[path+str(key)+'/sub_boxes'].value
            num = arr.shape[0]
            delete_list = [] # whose rows to be deleted
            # those rows should be deleted in reverse manner
            for i in range(num):
                obj_id = arr[i,0]
                sub_id = arr[i,2]
                obj_str = class_id2name_dict[obj_id]
                sub_str = class_id2name_dict[sub_id]
#                 if (obj_str in final_object_list) and (sub_str in final_object_list):
                if obj_str in final_object_list and sub_str in final_object_list:
                    continue
                else: delete_list.append(i)
            
            # delete rows in obj, arr, sub numpy array in reverse manner
            for i in reversed(delete_list):
                obj = np.delete(obj, i, axis=0)
                arr = np.delete(arr, i, axis=0)
                sub = np.delete(sub, i, axis=0)
            # if every bounding box is deleted, abandon this image
            if obj.size==0:
                del f[path+str(key)]
                deleted = deleted + 1
#                 print("deleted", key)
                continue
            # clear identical rows in hstacked np array
            total = np.hstack((obj,arr,sub))
            delete_list = set()
            for i in range(total.shape[0]): #generate pairs
                for j in range(i+1,total.shape[0]): 
                    if np.array_equal(total[i],total[j]): #compare rows
                        delete_list.add(j)
            delete_list = list(delete_list)
            delete_list.sort()
            for i in reversed(delete_list):
                total = np.delete(total, i, axis=0)
            # delete rows in delete_list in reverse order
            # slice back to the three arrays
            obj = total[:,:4]
            arr = total[:,4:7]
            sub = total[:,7:]
            # if this image is kept, update it's keys
#             print("saved", key)
            del f[path+str(key)+'/obj_boxes']
            del f[path+str(key)+'/rlp_labels']
            del f[path+str(key)+'/sub_boxes']
            f.create_dataset(path+str(key)+'/obj_boxes', data=obj)
            f.create_dataset(path+str(key)+'/rlp_labels', data=arr)
            f.create_dataset(path+str(key)+'/sub_boxes', data=sub)

In [223]:
paths=['/gt/test','/gt/train', '/meta/cls','/meta/imid2path','/meta/pre']
fs = h5py.File('filtered_vg.h5', 'r')
fd = h5py.File('both_filtered_no_recurrence_vg.h5','w')
for path in paths:
    group_path = fs[path].parent.name
    group_id = fd.require_group(group_path)
    fs.copy(path, group_id, name=path.split('/')[-1])

In [224]:
for path in paths:
    print(fd[path])

<HDF5 group "/gt/test" (10633 members)>
<HDF5 group "/gt/train" (30175 members)>
<HDF5 group "/meta/cls" (2 members)>
<HDF5 group "/meta/imid2path" (108077 members)>
<HDF5 group "/meta/pre" (2 members)>


In [225]:
fs.close()
fd.close()

In [212]:
with h5py.File('vg1_2_meta.h5','r') as f:
    print(f['/gt/test/1591925/rlp_labels'].value)

[[122  54 131]]


In [215]:
s=set()
s.add(np.array([1,2,3]))
print(s)
s.add(np.array([1,2,3]))
print(s)

TypeError: unhashable type: 'numpy.ndarray'

In [219]:
a=np.array([[1,2],[3,4]])
b=np.array([[3,4,5],[5,6,7]])
c=np.array([[5,7],[8,9]])
d=np.hstack((a,b,c))
print(d)
print(d[:,:2])
print(d[:,2:5])
print(d[:,5:])

[[1 2 3 4 5 5 7]
 [3 4 5 6 7 8 9]]
[[1 2]
 [3 4]]
[[3 4 5]
 [5 6 7]]
[[5 7]
 [8 9]]


In [220]:
s=set()
s.add(5)
s.add(6)
print(list(s))

[5, 6]
